# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the players total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game-try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [1]:
import random

In [2]:
SUITS = ['H', 'C', 'D', 'S']
RANKS = '2 3 4 5 6 7 8 9 J Q K A'.split()
MAX_PLAYERS = 8
MAX_BALANCE = 1000
chip_balance = 0

In [3]:
RANKS

['2', '3', '4', '5', '6', '7', '8', '9', 'J', 'Q', 'K', 'A']

In [4]:
class Card(object):
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.is_ace = self.rank == 'A'
        
        if rank == 'A':
            self.point = 11
        elif rank in ['J', 'Q', 'K']:
            self.point = 10
        else:
            self.point = int(rank)
            
        self.hidden = False
    
    def __str__(self):
        if self.hidden:
            return '[X]'
        else:
            return '['+self.suit + ' ' + self.rank + ']'
    
    def hide_card(self):
        self.hidden = True
        
    def reveal_card(self):
        self.hidden = False


In [5]:
card = Card('H', 'A')
card.is_ace

True

In [6]:
class Deck(object):
    
    def __init__(self):
        self.cards = [Card(suit, rank) for suit in SUITS for rank in RANKS]
    
    def __str__(self):
    
        deck_string = ' '.join(str(card) for card in self.cards)
        return deck_string
    
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal_card(self):
        card = self.cards.pop()
        return card
    

In [7]:
deck = Deck()
deck.shuffle()
print(deck)
print(deck.deal_card())

[C A] [D 9] [S J] [C 8] [D J] [S Q] [D 6] [H 3] [D 8] [C 3] [H K] [D Q] [H 7] [C 6] [S K] [H 2] [S 8] [C K] [D K] [S 7] [S 2] [C Q] [S A] [D 2] [S 9] [H 6] [H J] [D 3] [S 6] [H Q] [H 8] [S 4] [H A] [C J] [C 4] [S 3] [H 4] [C 9] [D 4] [D 5] [C 5] [C 2] [S 5] [D A] [H 9] [C 7] [D 7] [H 5]
[H 5]


In [8]:
class Hand(object):
    def __init__(self):
        self.hand = []
        
    def add_card(self, card):
        self.hand.append(card)
        return self.hand
    
    def get_value(self):
        aces = 0
        value = 0
        
        value = sum(card.point for card in self.hand)
        aces = sum(card.is_ace for card in self.hand)
        
        while (value > 21) and aces:
            value -= 10
            aces -= 1
            
        return value
    
    def show_hand(self):
        for card in self.hand:
            print(card, end = ' ')


In [9]:
hand = Hand()
hand.add_card(deck.deal_card())
hand.add_card(deck.deal_card())
hand.hand
hand.get_value()
hand.show_hand()
hand.hand[1]
hand.show_hand()

[D 7] [C 7] [D 7] [C 7] 

In [10]:
class Dealer(object):
    def __init__(self, name, deck, hand):
        self.name = name
        self.deck = deck
        self.hand = hand
        self.is_bust = False
        
    def hit(self):
        print('Hitting....')
        self.hand.add_card(self.deck.deal_card())
        return self.hand
    
    def stand(self):
        print("{} gets {}. Done.".format(self.name, self.hand.get_value()))
        
    def check_bust(self):
        if self.hand.get_value() > 21:
            self.is_bust = True
            print("{} gets bust!".format(self.name))
        else:
            self.stand()
        

In [11]:
d = Dealer("Greg", deck, hand)

In [12]:
d.hit()

Hitting....


In [13]:
d.check_bust()

Greg gets bust!


In [14]:
class Player(Dealer):
    def __init__(self, name, deck, hand, bet= 10):
        Dealer.__init__(self, name, deck, hand)
        self.bet = bet
        self.is_surrender = False
        self.is_split = False
        self.split = []
        


In [15]:
p = Player("Percy", deck, hand)

In [16]:
p.hit()

Hitting....


In [17]:
p.hand.show_hand()

[D 7] [C 7] [H 9] [D A] 

In [18]:
p.hand.get_value()

24

In [19]:
p.hand

In [20]:
def input_func(prompt, type_=None, min_=None, max_=None, range_=None):
    value = ''
    while True:
        value = input(prompt)
        if type_ is not None:
            try:
                value = type_(value)
            except ValueError:
                print ("Sorry I don't understand.")
                continue
        if min_ is not None and value < min_:
            print ("Sorry your input can not be less than %d!" % min_)
        elif max_ is not None and value > max_:
            print ("Sorry your input can not be more than %d!" % max_)
        elif range_ is not None and value not in range_:
            print ("You must select from", range_)
        else:
            break
    return value

In [21]:
def play(player, deck):
    print (player.name + ': '), player.hand.show_hand()
    print ("Hand Value: {}".format(player.hand.get_value()))
    if player.name == 'Dealer':
        while player.hand.get_value() < 17:
            player.hit()
            player.hand.show_hand()
        player.check_bust()
    else:
        global chip_balance
        if chip_balance > player.bet and not player.is_split:
            if player.hand.hand[0].point == player.hand.hand[1].point:
                choice = input_func("Hit, Stand, DoubleDown, Split or Surrender? (h/s/d/p/u) ", str.lower,
                                    range_=('h', 's', 'd', 'p', 'u'))
            else:
                choice = input_func("Hit, Stand, DoubleDown or Surrender? (h/s/d/u) ", str.lower,
                                    range_=('h', 's', 'd', 'u'))
        else:
            choice = input_func("Hit, Stand or Surrender? (h/s/u) ", str.lower, range_=('h', 's', 'u'))
        while choice == 'h':
            player.hit()
            player.hand.show_hand()
            print ("Hand Value: {}".format(player.hand.get_value()))
            if player.hand.get_value() > 21:
                player.is_bust = True
                print ("%s gets bust!" % player.name)
                break
            choice = input_func("Hit or Stand? (h/s) ", str.lower, range_=('h', 's'))

        if choice == 's':
            player.stand()

        if choice == 'd':
            chip_balance -= player.bet
            player.bet *= 2
            print ("New balance = %d" % chip_balance)
            player.hit()
            player.hand.show_hand()
            print ("Hand Value: {}".format(player.hand.get_value()))
            player.check_bust()

        if choice == 'u':
            player.is_surrender = True
            chip_balance += (player.bet - player.bet / 2)
            print ("New balance = %d" % chip_balance)

        if choice == 'p':
            chip_balance -= player.bet
            print ("New balance = %d" % chip_balance)
            player.split.append(Player(' Split_1', deck, player.bet))
            player.split.append(Player(' Split_2', deck, player.bet))
            for p in player.split:
                p.hand.add_card(player.hand.pop(0))
                p.hand.add_card(deck.deal_card())
                p.is_split = True
                play(p, deck)


In [22]:
def report(player, dealer):
    global chip_balance
    if player.is_surrender:
        tag = 'surrender'
    elif player.is_bust:
        tag = 'lose'
    elif len(player.hand.hand) == 2 and player.hand.get_value() == 21 and not player.is_split:
        tag = 'blackjack'
        chip_balance += player.bet * 3
    elif dealer.is_bust or (player.hand.get_value() > dealer.hand.get_value()):
        tag = 'win'
        chip_balance += player.bet * 2
    elif player.hand.get_value() == dealer.hand.get_value():
        tag = 'push'
        chip_balance += player.bet
    else:
        tag = 'lose'
    print ("%s: %-*s Balance = %d" % (player.name, 10, tag, chip_balance))


In [23]:
def game():
    players = []
    global chip_balance
    deck = Deck()
    deck.shuffle()

    player_num = input_func("\nPlease enter the number of players: (1-8) ", int, 1, MAX_PLAYERS)

    print ("\nLet's get started...\n")

    for i in range(player_num):
        if chip_balance > 0:
            player_name = 'Player_' + str(i + 1)
            print ("%s:" % player_name)
            player_bet = input_func("Please bet. The minimal bet is 1 chip. ", int, 1, chip_balance)
            chip_balance -= player_bet
            print ("Balance updated. New balance is %d." % chip_balance)
            player = Player(player_name, deck, Hand(), player_bet)
            players.append(player)
        else:
            print ("\nThe actual number of player is %d. There's no balance to support more players." % (len(players)))
            break

    dealer = Dealer('Dealer', deck, Hand())

    for i in range(2):
        for player in (players + [dealer]):
            player.hand.add_card(deck.deal_card())

    dealer.hand.hand[1].hide_card()
    print ("\nDealer:")
    dealer.hand.show_hand()
    print("\n")
    dealer.hand.hand[1].reveal_card()

    for player in (players + [dealer]):
        play(player, deck)
        print("\n")

    print ("...Final result...\n")

    for player in players:
        if not player.split:
            report(player, dealer)
        else:
            print ("%s: split" % player.name)
            for p in player.split:
                report(p, dealer)

    print ("\nFinal chip balance is %d.\n" % chip_balance)


In [24]:
if __name__ == '__main__':

    chip_balance = input_func("\nWelcome to BlackJack! Please enter the chip balance: (1-1000) ", int, 1, MAX_BALANCE)
    while True:
        game()
        if chip_balance < 1:
            print ("You don't have enough balance to proceed. Game over.")
            break
        proceed = input_func("Do you want to continue? (y/n) ", str.lower, range_=('y', 'n'))
        if proceed == 'n':
            print ("\nThank you for playing! See you next time.")
            break


Welcome to BlackJack! Please enter the chip balance: (1-1000) 100

Please enter the number of players: (1-8) 2

Let's get started...

Player_1:
Please bet. The minimal bet is 1 chip. 10
Balance updated. New balance is 90.
Player_2:
Please bet. The minimal bet is 1 chip. 10
Balance updated. New balance is 80.

Dealer:
[H A] [X] 

Player_1: 
[S 6] [S K] Hand Value: 16
Hit, Stand, DoubleDown or Surrender? (h/s/d/u) h
Hitting....
[S 6] [S K] [C K] Hand Value: 26
Player_1 gets bust!


Player_2: 
[D A] [H 6] Hand Value: 17
Hit, Stand, DoubleDown or Surrender? (h/s/d/u) s
Player_2 gets 17. Done.


Dealer: 
[H A] [D 3] Hand Value: 14
Hitting....
[H A] [D 3] [H 9] Hitting....
[H A] [D 3] [H 9] [S 8] Dealer gets 21. Done.


...Final result...

Player_1: lose       Balance = 80
Player_2: lose       Balance = 80

Final chip balance is 80.

Do you want to continue? (y/n) y

Please enter the number of players: (1-8) 2

Let's get started...

Player_1:
Please bet. The minimal bet is 1 chip. 20
Balanc